## [Working Title]

Created by Toomas Erik Anijärv in 25.05.2023

This notebook is a representation of EEG processing done for the publication with one of the participants as an example.

You are free to use this or any other code from this repository for your own projects and publications. Citation or reference to the repository is not required, but would be much appreciated (see more on README.md).

In [ ]:
# Import packages
import mne, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import seaborn as sns
from autoreject import (get_rejection_threshold, AutoReject)
from fooof import FOOOF
from fooof.plts.spectra import plot_spectrum, plot_spectrum_shading

# Set the default directory
os.chdir('/Users/tanijarv/Documents/GitHub/EEG-pyline')
mne.set_log_level('error')

# Import functions
import basic.arrange_data as arrange
import signal_processing.pre_process as prep
import signal_processing.spectral_analysis as spectr
import signal_processing.erp_analysis as erpan

In [ ]:
# Folder where to get the raw EEG files
raw_folder = 'Data/Raw/'

# Folder where to export the clean epochs files
clean_folder = 'Data/Clean/'

# Folder where to save the results and plots
results_folder = 'Results/'

# Sub-folder for the experiment (i.e. timepoint or group)
exp_folder = 'LEISURE/T1/SART/'
exp_condition = 'SART_T1'

### PRE-PROCESSING & TASK PERFORMANCE

In [73]:
# EOG + mastoid channels and stimulus channel
eog_channels = ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8']
stimulus_channel = 'Status'

# Parameters for filter design
filter_design = dict(l_freq=1, h_freq=30, filter_length='auto', method='fir',
                     l_trans_bandwidth='auto', h_trans_bandwidth='auto',
                     phase='zero', fir_window='hamming', fir_design='firwin')

# Epoch time window from event/stimuli
tminmax = [-0.2, 1]

# Baseline correction time window
baseline_correction = None

# Event names with IDs for GO and NO-GO trials
event_dict = {'GO trial': 4,
              'NO-GO trial': 8}

# Button press ID
button_id = 16

In [ ]:
# Get directories of raw EEG files and set export directory for clean files
dir_inprogress = os.path.join(raw_folder,exp_folder)
export_dir = os.path.join(clean_folder,exp_folder)
file_dirs, subject_names = arrange.read_files(dir_inprogress,'.bdf')

In [ ]:
# Loop through all the subjects' directories (EEG files directories)
df_success = pd.DataFrame()
for i in range(len(file_dirs)):
        print('\n{} ({} / {})'.format(subject_names[i], i+1, len(file_dirs)))
        # Read in the raw EEG data
        try:
                raw = mne.io.read_raw_bdf(file_dirs[i], infer_types=True, eog=eog_channels,
                                        stim_channel=stimulus_channel).drop_channels(['Erg1'])
        except:
                raw = mne.io.read_raw_bdf(file_dirs[i], infer_types=True, eog=eog_channels,
                                        stim_channel=stimulus_channel)

        # Set the right montage (Biosemi32) and set reference as average across all channels
        raw = raw.set_montage(mne.channels.make_standard_montage('biosemi32')).load_data()\
                 .set_eeg_reference(ref_channels='average', verbose=False)
    
        # Filter the signal with bandpass filter and remove EOG artefacts with SSP
        filt = prep.filter_raw_data(raw,filter_design, line_remove=None, eog_channels=eog_channels,
                                    plot_filt=False, savefig=False, verbose=False)
        
        # Find events from the filtered EEG data and name them
        events = mne.find_events(filt, stim_channel=stimulus_channel, consecutive=False, output='onset')
        
        # Create an array and dataframe of successful GO (followed with button press) and NO-GO trials (no button press)
        success_go_nogo_events = []
        df_success_temp = pd.DataFrame(index=[subject_names[i]],
                                       data={'Total GO' : np.sum(events[:, 2] == event_dict['GO trial']),
                                             'Total NO-GO' : np.sum(events[:, 2] == event_dict['NO-GO trial']),
                                             'Correct GO': 0, 'Correct NO-GO': 0})
        for m in range(len(events)):
                if events[m][2] == event_dict['GO trial'] and events[m+1][2] == button_id:
                        success_go_nogo_events.append(events[m])
                        df_success_temp['Correct GO'] += 1
                if events[m][2] == event_dict['NO-GO trial'] and events[m+1][2] != button_id:
                        success_go_nogo_events.append(events[m])
                        df_success_temp['Correct NO-GO'] += 1
        success_go_nogo_events = np.asarray(success_go_nogo_events)
        df_success_temp['Incorrect GO'] = df_success_temp['Total GO'] - df_success_temp['Correct GO']
        df_success_temp['Incorrect NO-GO'] = df_success_temp['Total NO-GO'] - df_success_temp['Correct NO-GO']
        df_success = pd.concat([df_success, df_success_temp])
        
        # Plot all the events
        %matplotlib inline
        fig, axs = plt.subplots(1, 1, figsize=(10, 5))
        fig = mne.viz.plot_events(success_go_nogo_events, sfreq=filt.info['sfreq'],
                                  first_samp=filt.first_samp, event_id=event_dict,
                                  axes=axs, show=False)
        fig.subplots_adjust(right=0.7)
        axs.set_title('Successful events ({})'.format(subject_names[i]))
        plt.show()

        # Create epochs time-locked to successful GO and NO-GO events
        picks = mne.pick_types(filt.info, eeg=True, stim=False)
        epochs = mne.Epochs(filt, success_go_nogo_events, event_id=event_dict,
                            tmin=tminmax[0], tmax=tminmax[1], baseline=baseline_correction,
                            picks=picks, preload=True)
        
        # Plot the epochs' GFP plot before artefact rejection
        epochs.plot_image(title="GFP without AR ({})".format(subject_names[i]))

        # Use AutoReject to repair and remove epochs which are artefactual
        reject_criteria = get_rejection_threshold(epochs)
        print('Dropping epochs with rejection threshold:',reject_criteria)
        epochs.drop_bad(reject=reject_criteria)

        ar = AutoReject(thresh_method='random_search', random_state=1)
        ar.fit(epochs)
        epochs_ar, reject_log = ar.transform(epochs, return_log=True)
        reject_log.plot('horizontal')

        # Plot the epochs' GFP after artefact rejection
        epochs_ar.average().plot()
        epochs_ar.plot_image(title="GFP with AR ({})".format(subject_names[i]))

        # Display the final epochs object meta-data
        display(epochs_ar)

        # Save the cleaned EEG file as .fif file
        try:
                os.makedirs(export_dir)
        except FileExistsError:
                pass
        try:
                mne.Epochs.save(epochs_ar, fname='{}/{}_clean-epo.fif'.format(export_dir,
                                                                              subject_names[i]),
                                                                              overwrite=True)
        except FileExistsError:
                pass

# Save the dataframe for task performance
df_success.to_excel('{}/{}/{}_task_performance.xlsx'.format(results_folder,exp_folder,exp_condition))

### SPECTRAL ANALYSIS: APERIODIC + THETA ACTIVITY

In [ ]:
# Brain regions and their channels /// do for Fz, Cz, Pz
ch = 'Pz'

# Power spectra estimation parameters
psd_params = dict(method='welch', fminmax=[1, 30], window='hamming', window_duration=1,
                  window_overlap=0, zero_padding=3, tminmax=[0, 1])

# FOOOF (specparam) model parameters
fooof_params = dict(peak_width_limits=[1,12], max_n_peaks=float('inf'), min_peak_height=0.225,
                    peak_threshold=2.0, aperiodic_mode='fixed')

# Band power of interest
bands = {'Theta' : [4, 8]}

# Flattened spectra amplitude scale (linear, log)
flat_spectr_scale = 'linear'

# Plot more information on the model fit plots or not; and save these plots or not
plot_rich = True
savefig = True

# Event names (i.e. different stimuli) within the epochs
event_list = ['GO trial', 'NO-GO trial']

In [ ]:
# Get directories of clean EEG files and set export directory
dir_inprogress = os.path.join(clean_folder, exp_folder)
file_dirs, subject_names = arrange.read_files(dir_inprogress, "_clean-epo.fif")

In [ ]:
# Pre-create results folders and dataframe
arrange.create_results_folders(exp_folder=exp_folder, results_folder=results_folder, fooof=True)
df_ch = pd.DataFrame()
# Go through all the files (subjects) in the folder
for i in range(len(file_dirs)):
    # Read the clean data from the disk
    epochs = mne.read_epochs(fname='{}/{}_clean-epo.fif'.format(dir_inprogress, subject_names[i]),
                                                                verbose=False)

    # Loop through all different events
    df_ch_ev = pd.DataFrame()
    df_ch_ev_diff = pd.DataFrame()
    for ev in event_list:
        print('{} for {} ({}/{})'.format(ev, subject_names[i], i+1, len(file_dirs)))

        ### POST-EVENT PSD ESTIMATION

        # Choose only epochs from the current event
        epochs_ev = epochs[ev]

        # Calculate Welch's power spectrum density (FFT) for the mean post-event
        [psds, freqs] = spectr.calculate_psd(epochs_ev, subject_names[i], **psd_params, verbose=True, plot=False)
        
        # Average all epochs and channels together -> (freq bins,) shape
        if i == 0:
            psds_allch = np.zeros(shape=(len(file_dirs), len(freqs)))
        psds_allch[i] = psds.mean(axis=(0, 1))

        # Average all epochs together for each channel and also for each region
        psds = psds.mean(axis=(0))
        df_psds_ch = arrange.array_to_df(subject_names[i], epochs_ev, psds).\
                            reset_index().drop(columns='Subject')

        # Choose only channel of interest data
        psds_temp = df_psds_ch[ch].to_numpy()

        ### ERP & POST-minus-ERP PSD ESTIMATIONS

        # Average the event epochs in time domain
        evoked_ev = epochs_ev.average(picks=ch)

        # Calculate Welch's power spectrum density (FFT) for the ERP
        [psds_erp, freqs] = spectr.calculate_psd(evoked_ev, subject_names[i], **psd_params, verbose=True, plot=False)

        # Calculate the post-minus-ERP PSD subtracting ERP PSD from post-event PSD
        psds_diff_ch = psds_temp - psds_erp[0]

        ### SPECPARAM

        # Fit the spectrums with FOOOF
        fm = FOOOF(**fooof_params, verbose=True)
        fm.fit(freqs, psds_temp, psd_params['fminmax'])
        fm_diff = FOOOF(**fooof_params, verbose=True)
        fm_diff.fit(freqs, psds_diff_ch, psd_params['fminmax'])
            
        # Log-linear conversion based on the chosen amplitude scale
        if flat_spectr_scale == 'linear':
            flatten_spectrum = 10 ** fm._spectrum_flat
            flatten_spectrum_diff = 10 ** fm_diff._spectrum_flat
            flat_spectr_ylabel = 'Amplitude (uV\u00b2/Hz)'
        elif flat_spectr_scale == 'log':
            flatten_spectrum = fm._spectrum_flat
            flatten_spectrum_diff = fm_diff._spectrum_flat
            flat_spectr_ylabel = 'Log-normalised amplitude'

        # Find individual alpha band parameters
        abs_bp, rel_bp = spectr.find_bp(flatten_spectrum, freqs, bands['Theta'])
        abs_bp_diff, rel_bp_diff = spectr.find_bp(flatten_spectrum_diff, freqs, bands['Theta'])

        ### PLOTTING

        # Set plot styles
        data_kwargs = {'color' : 'black', 'linewidth' : 1.4, 'label' : 'Original'}
        model_kwargs = {'color' : 'red', 'linewidth' : 1.4, 'alpha' : 0.75, 'label' : 'Full model'}
        aperiodic_kwargs = {'color' : 'blue', 'linewidth' : 1.4, 'alpha' : 0.75,
                            'linestyle' : 'dashed', 'label' : 'Aperiodic model'}
        flat_kwargs = {'color' : 'black', 'linewidth' : 1.4}
        hvline_kwargs = {'color' : 'blue', 'linewidth' : 1.0, 'linestyle' : 'dashed', 'alpha' : 0.75}
        
        # Plot power spectrum model + aperiodic fit for MEAN POST-EVENT PSD
        fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), dpi=100)
        plot_spectrum(fm.freqs, fm.power_spectrum,
                    ax=axs[0], plot_style=None, **data_kwargs)
        plot_spectrum(fm.freqs, fm.fooofed_spectrum_,
                    ax=axs[0], plot_style=None, **model_kwargs)
        plot_spectrum(fm.freqs, fm._ap_fit,
                    ax=axs[0], plot_style=None, **aperiodic_kwargs)
        axs[0].set_xlim(psd_params['fminmax'])
        axs[0].grid(linewidth=0.2)
        axs[0].set_xlabel('Frequency (Hz)')
        axs[0].set_ylabel('Log-normalised power (log$_{10}$[µV\u00b2/Hz])')
        axs[0].set_title('Spectrum model fit')
        axs[0].legend()
        
        # Flattened spectrum plot (i.e., minus aperiodic fit)
        plot_spectrum_shading(fm.freqs, flatten_spectrum,
                    ax=axs[1], shades=bands['Theta'], shade_colors='green',
                    plot_style=None, **flat_kwargs)
        #axs[1].vlines(bands['Theta'], ymin=axs[1].get_ylim()[0], ymax=axs[1].get_ylim()[1])
        axs[1].set_xlim(psd_params['fminmax'])
        axs[1].grid(linewidth=0.2)
        axs[1].set_xlabel('Frequency (Hz)')
        axs[1].set_ylabel(flat_spectr_ylabel)
        axs[1].set_title('Flattened spectrum')

        # If true, plot all the exported variables on the plots
        if plot_rich == True:
            axs[0].annotate('Error: ' + str(np.round(fm.get_params('error'), 4)) +
                        '\nR\u00b2: ' + str(np.round(fm.get_params('r_squared'), 4)),
                        (0.1, 0.16), xycoords='figure fraction', color='red', fontsize=8.5)
            axs[0].annotate('Exponent: ' + str(np.round(fm.get_params('aperiodic_params','exponent'), 4)) +
                        '\nOffset: ' + str(np.round(fm.get_params('aperiodic_params','offset'), 4)),
                        (0.19, 0.16), xycoords='figure fraction', color='blue', fontsize=8.5)
            axs[1].annotate('Absolute theta BP: '+str(np.round(abs_bp, 4))+'\nRelative theta BP: '+str(np.round(rel_bp, 4)),
                            (0.69, 0.16), xycoords='figure fraction', color='green', fontsize=8.5)
        
        plt.suptitle('Mean post-event PSD at {} ({})'.format(ch, subject_names[i]))
        plt.tight_layout()
        if savefig == True:
            plt.savefig(fname='{}/{}/FOOOF/{}_{}_{}_mean_post_event_PSD.png'.format(results_folder, exp_folder,
                                                                        exp_condition, subject_names[i],
                                                                        ch), dpi=300)
        plt.show()

        # Plot power spectrum model + aperiodic fit for POST-minus-ERP PSD
        fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), dpi=100)
        plot_spectrum(fm_diff.freqs, fm_diff.power_spectrum,
                    ax=axs[0], plot_style=None, **data_kwargs)
        plot_spectrum(fm_diff.freqs, fm_diff.fooofed_spectrum_,
                    ax=axs[0], plot_style=None, **model_kwargs)
        plot_spectrum(fm_diff.freqs, fm_diff._ap_fit,
                    ax=axs[0], plot_style=None, **aperiodic_kwargs)
        axs[0].set_xlim(psd_params['fminmax'])
        axs[0].grid(linewidth=0.2)
        axs[0].set_xlabel('Frequency (Hz)')
        axs[0].set_ylabel('Log-normalised power (log$_{10}$[µV\u00b2/Hz])')
        axs[0].set_title('Spectrum model fit')
        axs[0].legend()
        
        # Flattened spectrum plot (i.e., minus aperiodic fit)
        plot_spectrum_shading(fm_diff.freqs, flatten_spectrum_diff,
                    ax=axs[1], shades=bands['Theta'], shade_colors='green',
                    plot_style=None, **flat_kwargs)
        #axs[1].vlines(bands['Theta'], ymin=axs[1].get_ylim()[0], ymax=axs[1].get_ylim()[1])
        axs[1].set_xlim(psd_params['fminmax'])
        axs[1].grid(linewidth=0.2)
        axs[1].set_xlabel('Frequency (Hz)')
        axs[1].set_ylabel(flat_spectr_ylabel)
        axs[1].set_title('Flattened spectrum')

        # If true, plot all the exported variables on the plots
        if plot_rich == True:
            axs[0].annotate('Error: ' + str(np.round(fm_diff.get_params('error'), 4)) +
                        '\nR\u00b2: ' + str(np.round(fm_diff.get_params('r_squared'), 4)),
                        (0.1, 0.16), xycoords='figure fraction', color='red', fontsize=8.5)
            axs[0].annotate('Exponent: ' + str(np.round(fm_diff.get_params('aperiodic_params','exponent'), 4)) +
                        '\nOffset: ' + str(np.round(fm_diff.get_params('aperiodic_params','offset'), 4)),
                        (0.19, 0.16), xycoords='figure fraction', color='blue', fontsize=8.5)
            axs[1].annotate('Absolute theta BP: '+str(np.round(abs_bp_diff, 4))+'\nRelative theta BP: '+str(np.round(rel_bp_diff, 4)),
                            (0.69, 0.16), xycoords='figure fraction', color='green', fontsize=8.5)
        
        plt.suptitle('Post-minus-ERP PSD at {} ({})'.format(ch, subject_names[i]))
        plt.tight_layout()
        if savefig == True:
            plt.savefig(fname='{}/{}/FOOOF/{}_{}_{}_post_minus_erp_PSD.png'.format(results_folder, exp_folder,
                                                                        exp_condition, subject_names[i],
                                                                        ch), dpi=300)
        plt.show()

        ### EXPORTING

        # Add model parameters to dataframe for mean post-event
        df_ch_ev.loc[i, 'Exponent'] = fm.get_params('aperiodic_params','exponent')
        df_ch_ev.loc[i, 'Offset'] = fm.get_params('aperiodic_params','offset')
        df_ch_ev.loc[i, '{} absolute power'.format(list(bands.keys())[0])] = abs_bp
        df_ch_ev.loc[i, '{} relative power'.format(list(bands.keys())[0])] = rel_bp
        df_ch_ev.loc[i, 'R_2'] = fm.get_params('r_squared')
        df_ch_ev.loc[i, 'Error'] = fm.get_params('error')
        df_ch_ev['Channel'] = ch
        df_ch_ev['Event'] = ev
        df_ch_ev['Type'] = 'Mean post-event'
        df_ch_ev['Subject'] = subject_names[i]

        # Concatenate to master dataframe for mean post-event
        df_ch = pd.concat([df_ch, df_ch_ev])

        # Add model parameters to dataframe for post-minus-erp
        df_ch_ev_diff.loc[i, 'Exponent'] = fm_diff.get_params('aperiodic_params','exponent')
        df_ch_ev_diff.loc[i, 'Offset'] = fm_diff.get_params('aperiodic_params','offset')
        df_ch_ev_diff.loc[i, '{} absolute power'.format(list(bands.keys())[0])] = abs_bp_diff
        df_ch_ev_diff.loc[i, '{} relative power'.format(list(bands.keys())[0])] = rel_bp_diff
        df_ch_ev_diff.loc[i, 'R_2'] = fm_diff.get_params('r_squared')
        df_ch_ev_diff.loc[i, 'Error'] = fm_diff.get_params('error')
        df_ch_ev_diff['Channel'] = ch
        df_ch_ev_diff['Event'] = ev
        df_ch_ev_diff['Type'] = 'Post-minus-ERP'
        df_ch_ev_diff['Subject'] = subject_names[i]

        # Concatenate to master dataframe for post-minus-erp
        df_ch = pd.concat([df_ch, df_ch_ev_diff])
        

# Reorder the channels and reset index
df_ch = df_ch[['Subject', 'Channel', 'Type', 'Event', 'Exponent', 'Offset',
               '{} absolute power'.format(list(bands.keys())[0]),
               '{} relative power'.format(list(bands.keys())[0]),
               'R_2', 'Error']]
df_ch = df_ch.reset_index(drop=True)

# Export results for post-event data
df_ch.to_excel('{}/{}/FOOOF/{}_{}_specparam.xlsx'.format(results_folder, exp_folder, exp_condition, ch))
display(df_ch)

### ERP DETECTION & IDENTIFICATION

In [ ]:
tminmax = [-200, 1000]

# Time windows for different ERP components
erp_wins = {'N1' : [40, 170, -1],
            'N2' : [180, 350, -1],
            'P2' : [100, 260, 1],
            'P3' : [270, 500, 1]}

# Channel of interest
channel_picks = 'Pz'

# Event names (i.e. different stimuli) within the epochs
event_list = ['GO trial', 'NO-GO trial']

In [ ]:
# Get directories of clean EEG files and set export directory
dir_inprogress = os.path.join(clean_folder, exp_folder)
file_dirs, subject_names = arrange.read_files(dir_inprogress, '_clean-epo.fif')

In [ ]:
# Loop through all the subjects' directories (EEG files directories)
df_erps = pd.DataFrame()
arrange.create_results_folders(exp_folder=exp_folder,results_folder=results_folder, erps=True)
for i in range(len(file_dirs)):
    erp_wins_temp = erp_wins
    # Read the clean data from the disk
    epochs = mne.read_epochs(fname='{}/{}_clean-epo.fif'.format(dir_inprogress, subject_names[i]), verbose=False)
    
    # Apply baseline correction
    epochs = epochs.apply_baseline(baseline=(None, 0))

    ### create loop here for going through GO and NO-GO's separately
    for ev in event_list:
        print('{} for {} ({}/{})'.format(ev, subject_names[i], i, len(file_dirs)))
        # Create an averaged evoked object from epochs
        evoked_signal = epochs[ev].average(picks=channel_picks)

        # remove or add if save_evoked === truuuu
        evoked_signal.save('{}/{}/ERP analysis/{}_{}_{}_evoked-ave.fif'.format(results_folder, exp_folder,
                                                                            subject_names[i], channel_picks,
                                                                            ev), overwrite=True)

        # Find all the peaks in the evoked signal
        minpeak_times, minpeak_mags, maxpeak_times, maxpeak_mags = erpan.find_all_peaks(evoked_signal, epochs, 
                                                                                        t_range=tminmax, thresh=None,
                                                                                        subject_name=subject_names[i],
                                                                                        verbose=False, plot=False)
        
        # Identify which peaks are which ERPs based on the pre-defined ERP time windows
        erp_peaks, not_erp_peaks = erpan.identify_erps(evoked_signal, erp_wins_temp, minpeak_times, minpeak_mags,
                                                    maxpeak_times, maxpeak_mags, t_range=tminmax, subject_name=subject_names[i],
                                                    verbose=False, plot=True, savefig=False,
                                                    results_foldername=results_folder, exp_folder=exp_folder)

        # After visual inspection, it's possible to re-define the time windows to look for the peak
        while input('Do you need to do any manual time window changes? (leave empty if "no")') != '':
            print('Changing time window parameters for {}'.format(subject_names[i]))
            new_time_win = [None, None, None]

            # Ask user for which ERP they want to change or add
            erp_tochange = input('What ERP time window you want to change (e.g., N1)?')

            # Ask user what should be the minimum timepoint of the time window for that ERP
            new_time_win[0] = int(input('Enter MIN time of the window in interest for {} (e.g., 50)'.format(erp_tochange)))

            # Ask user what should be the maximum timepoint of the time window for that ERP
            new_time_win[1] = int(input('Enter MAX time of the window in interest for {} (e.g., 100)'.format(erp_tochange)))

            # Ask user whether this ERP should be a postitive (1) or negative (-1) peak
            new_time_win[2] = int(input('Enter whether to look for MIN (-1) or MAX (1) voltage for {}'.format(erp_tochange)))

            # Change the temporary ERP time window parameters to the user inputted parameters
            erp_wins_temp[erp_tochange] = new_time_win
            print('Changing', erp_tochange, 'with new time window:', str(new_time_win))

            # Use these new parameters to find either minimum or maximum value in that range
            try:
                erp_peaks = erpan.find_minmax_erp(evoked_signal, erp_peaks, erp_tochange, new_time_win,
                                                t_range=tminmax, subject_name=subject_names[i], verbose=False, plot=True,
                                                savefig=False, results_foldername=results_folder, exp_folder=exp_folder)
            except:
                print('Something went wrong with manual ERP detection, try again.')

        # Add this/these new temporary ERP to the main dataframe
        df_erps_temp = erpan.erp_dict_to_df(erp_peaks, erp_wins_temp, subject_names[i])
        df_erps_temp['Event'] = ev
        df_erps_temp['Channel'] = channel_picks
        df_erps = pd.concat([df_erps, df_erps_temp])
        print('ERPs have been found and added to the dataframe for {}'.format(subject_names[i]))
        display(df_erps)

# Calculate relative peak-to-peak amplitudes between the ERPs
print('Adding relative amplitudes for N1-P2, P2-N2, N2-P3')
df_erps['N1-P2 amplitude'] = df_erps['P2 amplitude'] - df_erps['N1 amplitude']
df_erps['P2-N2 amplitude'] = df_erps['N2 amplitude'] - df_erps['P2 amplitude']
df_erps['N2-P3 amplitude'] = df_erps['P3 amplitude'] - df_erps['N2 amplitude']

# Export all the detected ERPs to an Excel spreadsheet
display(df_erps)
df_erps.to_excel('{}/{}/ERP analysis/{}_{}_grandaverage_erps.xlsx'.format(results_folder,exp_folder,exp_condition,channel_picks))

### DATA VISUALISATION: ERPs

In [ ]:
# Export the figure to results folder or not
savefig = True

# Subjects which to not plot
exclude_subjects = [] # ['OKTOS_0019', 'OKTOS_0024', 'OKTOS_0033']

# Channel of interest
ch = 'Pz'

# Event names (i.e. different stimuli) within the epochs
event_list = ['GO trial', 'NO-GO trial']

In [ ]:
sns.set_theme(context='notebook', font_scale=1.3,
              style='whitegrid', palette='muted',
              font='sans-serif')

# Get directories of clean EEG files and exclude the pre-defined subjects
dir_inprogress = os.path.join(clean_folder, exp_folder)
file_dirs, subject_names = arrange.read_files(dir_inprogress, "_clean-epo.fif",
                                      exclude_subjects=exclude_subjects)

# Loop through all the subjects' directories (EEG files directories)
evoked_signal_go = [None]*len(file_dirs)
evoked_signal_nogo = [None]*len(file_dirs)
for i in range(len(file_dirs)):
    # Read the clean data from the disk
    epochs = mne.read_epochs(fname='{}/{}_clean-epo.fif'.format(dir_inprogress,
                                                                subject_names[i]),
                                                                verbose=False)
    
    # Create an averaged evoked object from epochs for both events
    evoked_signal_go[i] = epochs['GO trial'].average(picks=ch)
    evoked_signal_nogo[i] = epochs['NO-GO trial'].average(picks=ch)

# Average all the averaged evoked objects, thereby creating a grand average signals
go_master_grand_evoked_data = mne.grand_average(evoked_signal_go).data[0]*1e6
go_master_grand_evoked_times = mne.grand_average(evoked_signal_go).times*1e3
nogo_master_grand_evoked_data = mne.grand_average(evoked_signal_nogo).data[0]*1e6
nogo_master_grand_evoked_times = mne.grand_average(evoked_signal_nogo).times*1e3

# Plot all experiments' grand average signals on a single plot
fig, ax = plt.subplots(figsize=(6, 4), layout='tight', dpi=150)
ax.plot(go_master_grand_evoked_times, go_master_grand_evoked_data, linewidth=3)
ax.plot(nogo_master_grand_evoked_times, nogo_master_grand_evoked_data, linewidth=3)
ax.legend(event_list)
ax.set_title('Grand average of all participants at {}'.format(ch))
ax.set_xlim([-200, 1000])
ax.yaxis.set_major_locator(MultipleLocator(1))
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Amplitude (µV)')
ax.grid(which='major', axis='y', alpha=0.2)
ax.grid(which='major', axis='x', alpha=0.7)
if savefig == True:
    plt.savefig(fname='{}/{}/GRAND_erpfig_{}.png'.format(results_folder, exp_folder, ch),
                dpi=300)
plt.show()